### Interactive Model Demo

This notebook is an interactive demo of the trained models. \
You can select the model you want to use and then draw a digit or number for the model to predict.

In [ ]:
# Install dependencies
%pip install --upgrade pip
%pip install numpy
%pip install ipympl
%pip install ipywidgets
%pip install ipycanvas
%pip install matplotlib
%pip install tensorflow[and-cuda]
%pip install opencv-python-headless

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipycanvas import Canvas, hold_canvas  # type: ignore
from ipywidgets import Button, VBox, HBox, Output, Label, Dropdown
from tensorflow.keras.models import load_model  # type: ignore
import cv2

# Load Model
class ModelHandler:
    def __init__(self, model_path):
        self.model = load_model(model_path)

    def predict(self, image):
        """Run prediction on the image using the loaded model."""
        image = cv2.resize(image, (28, 28))
        image = image.astype('float32') / 255.0
        image = np.expand_dims(image, axis=-1)
        image = np.expand_dims(image, axis=0)
        return self.model.predict(image)

# Drawing and Prediction Class
class App:
    def __init__(self, model_paths):
        self.canvas_size = 28
        self.scale = 10
        self.canvas_data = np.zeros((self.canvas_size, self.canvas_size), dtype=np.uint8)
        self.canvas = Canvas(width=self.canvas_size * self.scale, height=self.canvas_size * self.scale)
        self.canvas.layout.width = f"{self.canvas_size * self.scale}px"
        self.canvas.layout.height = f"{self.canvas_size * self.scale}px"
        self.drawing = False
        self.prev_x = None
        self.prev_y = None

        # Initialize model paths and dropdown for model selection
        self.model_paths = model_paths
        self.model_dropdown = Dropdown(options=[(name, path) for name, path in model_paths.items()], layout={'width': '275px'})
        self.model_handler = ModelHandler(self.model_dropdown.value)
        self.class_labels = [
            '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 
            'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 
            'W', 'X', 'Y', 'Z', 'a', 'b', 'd', 'e', 'f', 'g', 'h', 'n', 'q', 'r', 't'
        ]

        # Initialize Buttons and Output
        self.clear_button = Button(description="Clear Canvas", button_style='danger', layout={'margin': '0 0 2px 11px'})
        self.predict_button = Button(description="Predict", button_style='success', layout={'margin': '2px 0 2px 11px'})
        self.output = Output()

        # Set up button actions
        self.clear_button.on_click(self.clear_canvas)
        self.predict_button.on_click(self.predict_and_display)
        self.model_dropdown.observe(self.on_model_change, names='value')

        # Attach canvas mouse events
        self.canvas.on_mouse_down(self.on_down)
        self.canvas.on_mouse_move(self.on_move)
        self.canvas.on_mouse_up(self.on_up)

        # Render initial black canvas
        self.render_canvas()

        # Add title text
        self.title_label = Label(
            value="Draw your own letter or digit and press 'Predict' to get the model's prediction.",
            style={'font_size': '13px', 'font_weight': '900'}
        )
        
        self.subtitle_label = Label(
            value="Make sure to use the full canvas area.",
            style={'font_size': '13px', 'font_weight': '900'}
        )

        # Display layout
        self.ui = VBox([
            self.title_label,
            self.subtitle_label,
            self.model_dropdown,
            HBox([self.canvas, VBox([self.clear_button, self.predict_button]), self.output])
        ])

    def on_model_change(self, change):
        """Reload model when a new model is selected from dropdown."""
        self.model_handler = ModelHandler(change['new'])

    def render_canvas(self):
        """Render the numpy array as the canvas background."""
        with hold_canvas(self.canvas):
            for y in range(self.canvas_size):
                for x in range(self.canvas_size):
                    color = self.canvas_data[y, x]
                    self.canvas.fill_style = f'rgba({color}, {color}, {color}, 1)'
                    self.canvas.fill_rect(x * self.scale, y * self.scale, self.scale, self.scale)

    def clear_canvas(self, _=None):
        """Clear the canvas."""
        self.canvas_data.fill(0)
        self.render_canvas()

    def on_down(self, x, y):
        """Start drawing when mouse is clicked."""
        self.drawing = True
        self.prev_x, self.prev_y = x, y
        self.draw(x, y)

    def on_move(self, x, y):
        """Draw when mouse is moved while the mouse is pressed down."""
        if self.drawing:
            self.draw(x, y)
    
    def on_up(self, x, y):
        """Stop drawing when mouse is released."""
        self.drawing = False
        self.prev_x, self.prev_y = None, None

    def draw(self, x, y):
        """Draw on the canvas with opacity around the drawn pixel."""
        grid_x, grid_y = int(x / self.scale), int(y / self.scale)
        if 0 <= grid_x < self.canvas_size and 0 <= grid_y < self.canvas_size:
            self.canvas_data[grid_y, grid_x] = 255
            self._draw_pixel(grid_x, grid_y, opacity=1.0)
            self._draw_surrounding_pixels(grid_x, grid_y)
            self.prev_x, self.prev_y = x, y

            with hold_canvas(self.canvas):
                self.canvas.fill_style = 'white'
                self.canvas.fill_rect(grid_x * self.scale, grid_y * self.scale, self.scale, self.scale)

    def _draw_pixel(self, x, y, opacity):
        """Draw a pixel with a given opacity."""
        if 0 <= x < self.canvas_size and 0 <= y < self.canvas_size:
            self.canvas_data[y, x] = int(255 * opacity)
            self.canvas.fill_style = f'rgba(255, 255, 255, {opacity})'
            self.canvas.fill_rect(x * self.scale, y * self.scale, self.scale, self.scale)

    def _draw_surrounding_pixels(self, x, y):
        """Draw surrounding pixels with reduced opacity."""
        surrounding_offsets = [(-1, 0), (1, 0), (0, -1), (0, 1), (-1, -1), (-1, 1), (1, -1), (1, 1)]
        for dx, dy in surrounding_offsets:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < self.canvas_size and 0 <= new_y < self.canvas_size:
                self._draw_pixel(new_x, new_y, opacity=0.7)

    def predict_and_display(self, _=None):
        """Run prediction on the drawn image and display in output widget."""
        predictions = self.model_handler.predict(self.canvas_data)
        class_probabilities = predictions[0] * 100
        high_probabilities = [(i, prob) for i, prob in enumerate(class_probabilities)]
        high_probabilities.sort(key=lambda x: x[1], reverse=True)

        # Table Data
        table_data = [(self.class_labels[idx], f"{prob:.2f}%") for idx, prob in high_probabilities[:10]]

        # Display Predictions
        with self.output:
            self.output.clear_output()
            fig, ax = plt.subplots(figsize=(4, 2))
            ax.axis("tight")
            ax.axis("off")
            ax.table(cellText=table_data, colLabels=["Class", "Probability"], cellLoc="center", loc="center")
            plt.show()

# Define available model paths
model_paths = {
    'EMNIST Balanced (89.0% Accuracy)': 'results/emnist-balanced/best-model.keras',
    'EMNIST Digits (99.6% Accuracy)': 'results/emnist-digits/best-model.keras',
    'MNIST Digits (99.5% Accuracy)': 'results/mnist/best-model.keras'
}

# Initialize app with model paths
app = App(model_paths=model_paths)

# Display the app layout
display(app.ui)